# Section 1: Loading and Examining some LFRic data

This Section uses output from LFRic and UM case study runs to show some basic differences of unstructured mesh (LFRic) and gridded data (UM).  

Example data is available with this tutorial.  
Let's dive right in by taking a look at some output file contents.

## Iris unstructured loading

"Unstructured" data can be loaded from UGRID files (i.e. netCDF files containing a UGRID-style mesh). This is just like normal [Iris](https://scitools-iris.readthedocs.io/en/latest) loading, except that we must *enable* the interpretion of UGRID content like this:

```python
with PARSE_UGRID_ON_LOAD.context():
    cube_list = iris.load(path [, constraints])
    # ..and/or..
    single_cube = iris.load_cube(path [, constraints])
    # ..and/or..
    selected_cubes = iris.load_cubes(path, cube_constraints)

```

### Enable UGRID loading

To test loading of UGRID files, like demonstrated above, we need to import `iris`

In [1]:
import iris

# This will suppress RuntimeWarning flags
import warnings
warnings.filterwarnings("ignore")

### Load UGRID data from netCDF files.
The tutorial helper code [testdata_fetching](./testdata_fetching.py) defines a variable `lfric_filepath` (and `um_filepath`) which points to suitable a test file.

In [2]:
# import tutorial helper routines for handling access to test data
from testdata_fetching import lfric_filepth, um_filepth
print('LFRic data availble from ' + str(lfric_filepth))
print('UM data availble from ' + str(um_filepth))

LFRic data availble from ../example_data/u-ct674_20210324T0000Z_lf_ugrid.nc
UM data availble from ../example_data/u-ct674_20210324T0000Z_um_latlon.nc


In this case, we use the plain [iris.load](https://scitools-iris.readthedocs.io/en/latest/userguide/loading_iris_cubes.html) function to load the data (but of course with the PARSE_UGRID_ON_LOAD context) from above listed files. 

We show the first few of the cubes. Putting just `cubes` at the end triggers noteboook printing output. You can click on each cube to expand it into detail view. Try this. Try also to use `print(cubes)` instead.

In [3]:
print('loading...')

iris.FUTURE.date_microseconds = True

cubes = iris.load(lfric_filepth)

print(f'\n... Loaded {len(cubes)} cubes.')
print('Showing first 6:')
cubes[:6]

loading...

... Loaded 9 cubes.
Showing first 6:


[<iris 'Cube' of surface_upward_longwave_flux_radiative_timestep / (W m-2) (time: 2; -- : 13824)>,
<iris 'Cube' of toa_upward_shortwave_flux / (W m-2) (time: 2; -- : 13824)>,
<iris 'Cube' of relative_humidity_wrt_water / (%) (time: 2; pressure: 4; -- : 13824)>,
<iris 'Cube' of relative_humidity_at_screen_level / (1) (time: 24; -- : 13824)>,
<iris 'Cube' of stable_boundary_layer_indicator / (1) (time: 2; -- : 13824)>,
<iris 'Cube' of relative_humidity_wrt_ice / (%) (time: 2; pressure: 4; -- : 13824)>]

Putting just `cubes` at the end of the code above triggers noteboook printing output. You can click on each cube to expand it into detail view. Try this. Try also to use `print(cubes)` instead. To spot some structual differences between LFRic and UM data also load some cubes from `um_filepath` above. 

## What is notable about "mesh cubes"

In the cube printout above, compared to regular UM-style data (try loading data from `um_filepth`), you can see that it has an additional section in the cube printout called "Mesh", which displays the mesh-specific info. The Mesh has "Mesh coordinates", containing information about latitude and longitude but no dimension coordinates for latitude or longitude. Cubes with a mesh are known in Iris as "unstructured cubes" or "mesh cubes". They also always have a specific "mesh dimension": In the above example it is the _last_ cube dimension.


The cube itself also now has some extra properties : `cube.mesh`, `cube.location` and `cube.mesh_dim()`  
(which are otherwise all `None` if the cube is not a mesh cube)


## Loading a single cube
To print the three mentioned extras properties we load just a single cube from the file (This is considerably faster if a file contains 100s variables). Let's load just `relative_humidity_wrt_water` data from the example file (`fric_filepth`) whith the [load_cube](https://scitools-iris.readthedocs.io/en/latest/generated/api/iris.html?highlight=load_cube#iris.load_cube) function and print those properties:

In [4]:
lfric_rh = iris.load_cube(lfric_filepth,'relative_humidity_wrt_water')

# just uncomment to explore: 
print(lfric_rh)
# print(lfric_rh.mesh)
# print(lfric_rh.location)
# print(lfric_rh.mesh_dim)


relative_humidity_wrt_water / (%)   (time: 2; pressure: 4; -- : 13824)
    Dimension coordinates:
        time                             x            -       -
        pressure                         -            x       -
    Mesh coordinates:
        latitude                         -            -       x
        longitude                        -            -       x
    Auxiliary coordinates:
        forecast_period                  x            -       -
    Mesh:
        name                        Topology data of 2D unstructured mesh
        location                    face
    Scalar coordinates:
        forecast_reference_time     2021-03-24 00:00:00
    Cell methods:
        0                           time: point
    Attributes:
        Conventions                 'CF-1.7'
        description                 'Created by xios'
        interval_operation          '6 h'
        interval_write              '6 h'
        online_operation            'instant'
        title    

If the cube is not a mesh cube these properties are `None`, which we can demonstrate with a cube from the "UM file":   

In [5]:
um_cube = iris.load_cube(um_filepth,'air_temperature')
print('You can see the data here: \n \n', um_cube)
print('\n and it is clearly not a mesh. \n \n If we try to print the mesh property, then the output is as follows: \n') # text to break up output
print(um_cube.mesh)


You can see the data here: 
 
 air_temperature / (K)               (time: 2; pressure: 4; latitude: 144; longitude: 192)
    Dimension coordinates:
        time                             x            -            -               -
        pressure                         -            x            -               -
        latitude                         -            -            x               -
        longitude                        -            -            -               x
    Auxiliary coordinates:
        forecast_period                  x            -            -               -
    Scalar coordinates:
        forecast_reference_time     2021-03-24 00:00:00
    Attributes:
        Conventions                 'CF-1.7'
        STASH                       m01s16i203
        source                      'Data from Met Office Unified Model'
        um_version                  '12.2'

 and it is clearly not a mesh. 
 
 If we try to print the mesh property, then the output is as 

## Next notebook
See the next section: [02 - Mesh concepts and Meshes in Iris](./Sec_02_Meshes.ipynb)